In [11]:
import netCDF4 as nc
import xarray as xr
from geoarray import GeoArray
from spectral.io import envi
import matplotlib.pyplot as plt
import numpy as np
from isofit.core.common import resample_spectrum
import os
import isofit
import tkinter
import matplotlib
matplotlib.use('TkAgg')
from isofit.core.sunposition import sunpos
import holoviews as hv
import hvplot.xarray
from datetime import datetime

In [12]:
obs_header = envi.open('../emit_obs_loc/emit20230321t150833_o08010_s000_l1b_obs_b0106_v01.hdr')
location_header = envi.open('../emit_obs_loc/emit20230321t150833_o08010_s000_l1b_loc_b0106_v01.hdr')
rad_header = envi.open('../west_africa_scene/wa_unortho/EMIT_L1B_RAD_001_20230321T150833_2308010_003_radiance.hdr')
mask_header = envi.open('../west_africa_scene/wa_unortho/EMIT_L2A_MASK_001_20230321T150833_2308010_003_mask.hdr')

obs_data = obs_header.open_memmap(interleave = 'bip')
loc_data = location_header.open_memmap(interleave = 'bip')
rad_data = rad_header.open_memmap(interleave = 'bip')
mask_data = mask_header.open_memmap(interleave = 'bip')

In [13]:
lat = loc_data[:,:,0]
lon = loc_data[:,:,1]
elev = loc_data[:,:,2]
zenith = obs_data[:,:,4]
zen = np.average(zenith)
es_distance = obs_data[:,:,10]
water_vapor = mask_data[:,:,6]

print('shape of latitude: ', lat.shape)
print('shape of longitude: ', lon.shape)
print('shape of elevation: ', elev.shape)
print('shape of zenith: ', zen.shape)
print('shape of earth-sun distance', es_distance.shape)
print('shape of water vapor: ', water_vapor.shape)

shape of latitude:  (1280, 1242)
shape of longitude:  (1280, 1242)
shape of elevation:  (1280, 1242)
shape of zenith:  ()
shape of earth-sun distance (1280, 1242)
shape of water vapor:  (1280, 1242)


In [14]:
wl = rad_header.metadata['wavelength']
for i in range(len(wl)):
    wl[i] = float(wl[i])
wl = np.array(wl)

fwhm = rad_header.metadata['fwhm']
for i in range(len(fwhm)):
    fwhm[i] = float(fwhm[i])
fwhm = np.array(fwhm)

# will need new irradiance file
irr_file = os.path.join(
    os.path.dirname(isofit.__file__), "..", "data", "kurudz_0.1nm.dat") # same for anything TOA, pure solar irradiance

print('path to irr file:', irr_file)


irr_wl, irr = np.loadtxt(irr_file, comments="#").T
irr = irr / 10  # convert to uW cm-2 sr-1 nm-1
irr_resamp = resample_spectrum(irr, irr_wl, wl, fwhm)
irr_resamp = np.array(irr_resamp, dtype=np.float32)
irr = irr_resamp

print('shape of wavelength: ', wl.shape)
print('shape of fwhm: ', fwhm.shape)
print('shape of irradiance: ', irr.shape)
print('shape of radiance: ', rad_data.shape)


path to irr file: C:\Users\vpatro\Desktop\isofit\isofit\..\data\kurudz_0.1nm.dat
shape of wavelength:  (285,)
shape of fwhm:  (285,)
shape of irradiance:  (285,)
shape of radiance:  (1280, 1242, 285)


In [15]:
flight_name = rad_header.metadata['flight_line'][0]

year = flight_name[4:8]
month = flight_name[8:10]
date = flight_name[10:12]
hour = flight_name[13:15]
minute = flight_name[15:17]
second = flight_name[17:19]

dt_string = ''
dt_string = dt_string + date + '/' + month + '/' + year + ' ' + hour + ':' + minute + ':' + second

dt = datetime.strptime(dt_string, "%d/%m/%Y %H:%M:%S")
print(dt)

2023-03-21 15:08:33


In [16]:
rho = (np.pi / np.cos(zen)) * rad_data / irr[np.newaxis, np.newaxis, :]
rho.shape

(1280, 1242, 285)

In [17]:
plt.plot(rho[750,750,:])
plt.show()

In [18]:
path_list = ['../emit_obs_loc/emit20230321t150833_o08010_s000_l1b_obs_b0106_v01.hdr',
             '../west_africa_scene/wa_unortho/EMIT_L1B_RAD_001_20230321T150833_2308010_003_radiance.hdr', 
             '../west_africa_scene/wa_unortho/EMIT_L2A_MASK_001_20230321T150833_2308010_003_mask.hdr']

irr_file = os.path.join(os.path.dirname(isofit.__file__), "..", "data", "kurudz_0.1nm.dat")

In [19]:
import preprocess

In [21]:
TOA_refl = preprocess.main(path_list, irr_file)

KeyError: 'wavelength'

In [22]:
TOA_refl.shape

NameError: name 'TOA_refl' is not defined

In [12]:
plt.plot(TOA_refl[750,750,:])
plt.show()